In [159]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import math

In [160]:
# import data
fixtures = pd.read_csv('fixtures.csv')

conditions = [
    fixtures['localteam_score'] > fixtures['visitorteam_score'],
    fixtures['visitorteam_score'] > fixtures['localteam_score'],
    fixtures['localteam_score'] == fixtures['visitorteam_score']
]
choices = [ 1, 0, 0.5 ]
fixtures['outcome'] = np.select(conditions, choices)

In [161]:
# we can use champions league to test; 
fixtures = fixtures[np.isnan(fixtures['localteam_id']) == False]
fixtures['localteam_id'] = fixtures['localteam_id'].apply(np.int)
fixtures = fixtures[fixtures['league_name'] == 'Champions League']

In [162]:
#rank fixtures played by each team
localteam_fixtures = fixtures[['localteam_id', 'fixture_id', 'starting_datetime']] \
                        .rename(index=str, columns={'localteam_id': 'team_id'})
visitorteam_fixtures = fixtures[['visitorteam_id', 'fixture_id', 'starting_datetime']] \
                        .rename(index=str, columns={'visitorteam_id': 'team_id'})
                          
frames = [localteam_fixtures, visitorteam_fixtures]
combined_fixtures = pd.concat(frames).reset_index()

combined_fixtures['fixture_order'] = combined_fixtures.groupby('team_id')['starting_datetime'].rank(ascending=True, method='min')
combined_fixtures = combined_fixtures.sort_values(['team_id','fixture_order'])
combined_fixtures['next_fixture'] = combined_fixtures.groupby('team_id')['fixture_id'].shift(-1)

In [163]:
fixtures = pd.merge(fixtures, combined_fixtures[['fixture_order', 'fixture_id', 'team_id', 'next_fixture']],\
                    how='left', left_on=['localteam_id','fixture_id'], right_on = ['team_id','fixture_id'])\
                .drop(['team_id'], axis=1)\
                .rename(index=str, columns={'fixture_order': 'localteam_order', 'next_fixture': 'localteam_next'})

fixtures = pd.merge(fixtures, combined_fixtures[['fixture_order', 'fixture_id', 'team_id', 'next_fixture']],\
            how='left', left_on=['visitorteam_id','fixture_id'], right_on = ['team_id','fixture_id'])\
        .drop(['team_id'], axis=1)\
        .rename(index=str, columns={'fixture_order': 'visitorteam_order', 'next_fixture': 'visitorteam_next'})

In [164]:
fixtures_f = fixtures

#fixtures_f.drop('cur_team_r', axis=1, inplace=True)
#fixtures_f.drop('opp_team_r', axis=1, inplace=True)
#fixtures_f.drop('cur_team_r', axis=1, inplace=True)
#fixtures_f.drop('opp_team_r', axis=1, inplace=True)
#fixtures_f.drop('cur_p', axis=1, inplace=True)
#fixtures_f.drop('opp_p', axis=1, inplace=True)
#fixtures_f.drop('cur_post_r', axis=1, inplace=True)
#fixtures_f.drop('opp_post_r', axis=1, inplace=True)

fixtures_f['localteam_r'] = np.nan
fixtures_f['visitorteam_r'] = np.nan
fixtures_f['localteam_post_r'] = np.nan
fixtures_f['visitorteam_post_r'] = np.nan

In [165]:
fixtures_f['localteam_r'].isnull().sum()

2028

In [166]:
def Probability(rating1, rating2):
    p1_w = 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating2 - rating1) / 400))
    p2_w = 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))
 
    return p1_w, p2_w

def EloRating(Ra, Rb, K, d):

    # Calculate winning probabilities
    Pa, Pb = Probability(Ra, Rb)
 
    # Case -1 When Player A wins
    # Updating the Elo Ratings
    if (d == 1) :
        Ra = Ra + K * (1 - Pa)
        Rb = Rb + K * (0 - Pb)    
 
    # Case -2 When Player B wins
    # Updating the Elo Ratings
    elif (d == 0.5):
        Ra = Ra + K * (0.5 - Pa)
        Rb = Rb + K * (0.5 - Pb)
        
    else:
        Ra = Ra + K * (0 - Pa)
        Rb = Rb + K * (1 - Pb)   
    
    return Pa, Pb, Ra, Rb
     
    ##    print("Updated Ratings:-")
    ##    print("Ra =", round(Ra, 6)," Rb =", round(Rb, 6))

In [ ]:
def test(data):
    K = 30
    
    nulls = data['localteam_r'].isnull().sum() + data['visitorteam_r'].isnull().sum()
    print(nulls)
    
    while nulls > 0:

        for index, row in data.iterrows():
            fixture_id = row['fixture_id']
            localteam_id = row['localteam_id']
            visitorteam_id = row['visitorteam_id']
            localteam_next = row['localteam_next']
            visitorteam_next = row['visitorteam_next']
            
#            if np.isnan(row['localteam_r']) == False & np.isnan(row['visitorteam_r']) == False & \
#                np.isnan(row['localteam_post_r']) == False & np.isnan(row['visitorteam_post_r']) == False:
#                return

            if (row['localteam_order'] == 1 & np.isnan(row['localteam_r'])) | (row['visitorteam_order'] == 1 & np.isnan(row['visitorteam_r'])):
                if row['localteam_order'] == 1 & np.isnan(row['localteam_r']):
                    data.loc[(data['fixture_id'] == fixture_id), 'localteam_r'] = 1500
                elif row['visitorteam_order'] == 1 & np.isnan(row['visitorteam_r']):
                    data.loc[(data['fixture_id'] == fixture_id), 'visitorteam_r'] = 1500
            
            elif np.isnan(row['localteam_r']) == False & np.isnan(row['visitorteam_r']) == False & \
                np.isnan(row['localteam_post_r']) & np.isnan(row['visitorteam_post_r']):
                    
                #print("evaluating fixture %d" % (fixture_id))
                
                #calculate ratings
                localteam_p, visitorteam_p, localteam_post_r, visitorteam_post_r = EloRating(row['localteam_r'], row['visitorteam_r'], K, row['outcome'])
                
                #assign probabilities for current game and post-game ratings
                data.loc[(data['fixture_id'] == fixture_id), 'localteam_p'] = localteam_p
                data.loc[(data['fixture_id'] == fixture_id), 'visitorteam_p'] = visitorteam_p
                data.loc[(data['fixture_id'] == fixture_id), 'localteam_post_r'] = localteam_post_r
                data.loc[(data['fixture_id'] == fixture_id), 'visitorteam_post_r'] = visitorteam_post_r
                
                #print('the localteam id is %d' % (localteam_id))
                #print('the visitorteam id is %d' % (visitorteam_id))
                
                #bring post-game ratings to next game
                if np.isnan(localteam_next) == False:    
                    l_next_localteam_id = data[data['fixture_id'] == localteam_next]['localteam_id'][0]
                    l_next_visitorteam_id = data[data['fixture_id'] == localteam_next]['visitorteam_id'][0]
                    
                    if localteam_id == 3444 | visitorteam_id == 3444:
                        print(fixture_id)
                        print(l_next_localteam_id)
                        print(l_next_visitorteam_id)
                        print(localteam_r)
                        print(visitorteam_r)
                    
                    if localteam_id == l_next_localteam_id:
                        data.loc[(data['fixture_id'] == localteam_next), 'localteam_r'] = localteam_post_r
                    else:
                        data.loc[(data['fixture_id'] == localteam_next), 'visitorteam_r'] = localteam_post_r
                    
                if np.isnan(visitorteam_next) == False:
                    v_next_localteam_id = data[data['fixture_id'] == visitorteam_next]['localteam_id'][0]
                    v_next_visitorteam_id = data[data['fixture_id'] == visitorteam_next]['visitorteam_id'][0]
                   
                    if visitorteam_id == v_next_localteam_id:
                        data.loc[(data['fixture_id'] == visitorteam_next), 'localteam_r'] = visitorteam_post_r
                    else:
                        data.loc[(data['fixture_id'] == visitorteam_next), 'visitorteam_r'] = visitorteam_post_r
                
                #print(l_next_localteam_id)
                #print(l_next_visitorteam_id)
                
        test(data)

In [ ]:
test(fixtures_f)

4056
3848
3796
3692
3612
3554
3506
3450
3408
3364
3328
3282
3235
3191
3153
3115
3079
3042
3012
2983
2952
2923
2889
2853
2813
2768
2729
2693
2660
2632
2599
2567
2540
2515
2491
2467
2443
2413
2383
2354
2329
2305
2277
2249
2224
2202
2178
2153
2123
2094
2064
2031
2005
1979
1958
1937
1913
1890
1868
1847
1821
1790
1748
1705
1660
1618
1585
1554
1527
1504
1484
1464
1437
1413
1383
1353
1322
1289
1257
1225
1193
1163
1127
1098
1066


In [150]:
fixtures_f[(fixtures_f['localteam_id']==3444) | (fixtures_f['visitorteam_id']==3444)].sort_values('starting_datetime')

,fixture_id,localteam_id,localteam_name,visitorteam_id,visitorteam_name,localteam_score,visitorteam_score,starting_datetime,league_name,country_name,...,localteam_order,localteam_next,visitorteam_order,visitorteam_next,localteam_r,visitorteam_r,localteam_post_r,visitorteam_post_r,localteam_p,visitorteam_p
1905,1060581,34,Galatasaray,3444,FCSB,2,2,2008-08-13 18:15:00,Champions League,Romania,...,1.0,1060604.0,1.0,1060604.0,1500.0000,1500.0000,1500.000000,1500.000000,0.500000,0.500000
1921,1060604,3444,FCSB,34,Galatasaray,1,0,2008-08-27 17:45:00,Champions League,Turkey,...,2.0,1058982.0,2.0,1058900.0,1500.0000,1500.0000,1515.000000,1485.000000,0.500000,0.500000
1944,1058982,3444,FCSB,503,Bayern München,0,1,2008-09-17 18:45:00,Champions League,Germany,...,3.0,1059046.0,1.0,1059038.0,1485.0000,1500.0000,1470.647200,1514.352800,0.478427,0.521573
1952,1059046,109,Fiorentina,3444,FCSB,0,0,2008-09-30 18:45:00,Champions League,Romania,...,4.0,1059183.0,4.0,1059172.0,1514.3528,1470.6472,1512.475768,1472.524232,0.562568,0.437432
1967,1059172,3444,FCSB,79,Olympique Lyonnais,3,5,2008-10-21 18:45:00,Champions League,France,...,5.0,1059377.0,5.0,1059377.0,NaN,NaN,NaN,NaN,NaN,NaN
1991,1059377,79,Olympique Lyonnais,3444,FCSB,2,0,2008-11-05 19:45:00,Champions League,Romania,...,6.0,1059441.0,6.0,1059448.0,NaN,NaN,NaN,NaN,NaN,NaN
2000,1059448,503,Bayern München,3444,FCSB,3,0,2008-11-25 19:45:00,Champions League,Romania,...,5.0,1059622.0,7.0,1059605.0,NaN,NaN,NaN,NaN,NaN,NaN
2017,1059605,3444,FCSB,109,Fiorentina,0,1,2008-12-10 19:45:00,Champions League,Italy,...,8.0,1061742.0,8.0,1059080.0,NaN,NaN,NaN,NaN,NaN,NaN
648,1061742,3444,FCSB,5492,Vardar,3,0,2013-07-16 17:45:00,Champions League,Macedonia FYR,...,9.0,1061752.0,1.0,1061752.0,NaN,1500.0000,NaN,NaN,NaN,NaN
668,1061752,5492,Vardar,3444,FCSB,1,2,2013-07-23 18:45:00,Champions League,Romania,...,2.0,1061617.0,10.0,1061298.0,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
fixtures_f[(fixtures_f['localteam_order'] == 2) | (fixtures_f['visitorteam_order'] == 2)]

,fixture_id,localteam_id,localteam_name,visitorteam_id,visitorteam_name,localteam_score,visitorteam_score,starting_datetime,league_name,country_name,...,localteam_order,localteam_next,visitorteam_order,visitorteam_next,localteam_r,visitorteam_r,localteam_post_r,visitorteam_post_r,localteam_p,visitorteam_p
3,10340559,10068,Newcastle FC,148038,Drita,1,1,2018-06-29 18:30:00,Champions League,Kosovo,...,12.0,NaN,2.0,10341793.0,NaN,NaN,NaN,NaN,NaN,NaN
7,10336090,2604,APOEL,5878,Sūduva,1,0,2018-07-17 17:00:00,Champions League,Lithuania,...,62.0,NaN,2.0,10362022.0,NaN,NaN,NaN,NaN,NaN,NaN
10,10336100,349,Legia Warszawa,326,Cork City,3,0,2018-07-17 19:00:00,Champions League,Republic of Ireland,...,28.0,10364237.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,10336112,6213,Zrinjski,3283,Spartak Trnava,1,1,2018-07-18 17:00:00,Champions League,Slovakia,...,10.0,NaN,2.0,10364237.0,NaN,NaN,NaN,NaN,NaN,NaN
36,10336086,627,Sheriff,2671,Torpedo Kutaisi,3,0,2018-07-18 17:00:00,Champions League,Georgia,...,32.0,10364270.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,1062118,5495,Alashkert,7058,FC Santa Coloma,3,0,2016-07-05 15:00:00,Champions League,Andorra,...,2.0,1061737.0,14.0,1818602.0,NaN,NaN,NaN,NaN,NaN,NaN
125,1061893,6142,Liepāja,49,Salzburg,0,2,2016-07-19 16:00:00,Champions League,Austria,...,2.0,NaN,22.0,1061486.0,NaN,NaN,NaN,NaN,NaN,NaN
126,1061791,627,Sheriff,516,Hapoel Be'er Sheva,0,0,2016-07-19 17:00:00,Champions League,Israel,...,26.0,1818634.0,2.0,1061695.0,NaN,NaN,NaN,NaN,NaN,NaN
128,1061832,826,SJK,3859,BATE,2,2,2016-07-19 17:00:00,Champions League,Belarus,...,2.0,NaN,60.0,1061424.0,NaN,NaN,NaN,NaN,NaN,NaN
130,1061854,2673,Crvena Zvezda,147,Valletta,2,1,2016-07-19 18:30:00,Champions League,Malta,...,2.0,1061400.0,12.0,10336103.0,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
fixtures_f[(fixtures_f['localteam_id']==7058) | (fixtures_f['visitorteam_id']==7058)].sort_values('starting_datetime')

,fixture_id,localteam_id,localteam_name,visitorteam_id,visitorteam_name,localteam_score,visitorteam_score,starting_datetime,league_name,country_name,...,localteam_order,localteam_next,visitorteam_order,visitorteam_next,localteam_r,visitorteam_r,localteam_post_r,visitorteam_post_r,localteam_p,visitorteam_p
1851,1061438,7058,FC Santa Coloma,11333,Kaunas,1,4,2008-07-15 18:30:00,Champions League,Lithuania,...,1.0,1061447.0,1.0,1061447.0,1500.0,1500.0,1485.000000,1515.000000,0.500000,0.500000
1859,1061447,11333,Kaunas,7058,FC Santa Coloma,3,1,2008-07-22 16:30:00,Champions League,Andorra,...,2.0,1061258.0,2.0,1061926.0,1515.0,1485.0,1528.708005,1471.291995,0.543066,0.456934
1063,1061926,7058,FC Santa Coloma,1877,Birkirkara,0,3,2010-06-29 17:00:00,Champions League,Malta,...,3.0,1061927.0,1.0,1061927.0,NaN,1500.0,NaN,NaN,NaN,NaN
1065,1061927,1877,Birkirkara,7058,FC Santa Coloma,4,3,2010-07-06 16:00:00,Champions League,Andorra,...,2.0,1061620.0,4.0,1061935.0,NaN,NaN,NaN,NaN,NaN,NaN
850,1061935,7058,FC Santa Coloma,1401,F91 Dudelange,0,2,2011-06-28 17:00:00,Champions League,Luxembourg,...,5.0,1061945.0,5.0,1061945.0,NaN,NaN,NaN,NaN,NaN,NaN
852,1061945,1401,F91 Dudelange,7058,FC Santa Coloma,2,0,2011-07-05 16:30:00,Champions League,Andorra,...,6.0,1061625.0,6.0,207684.0,NaN,NaN,NaN,NaN,NaN,NaN
423,207684,7058,FC Santa Coloma,5487,Banants,1,0,2014-07-01 18:00:00,Champions League,Armenia,...,7.0,1061948.0,1.0,1061948.0,NaN,1500.0,NaN,NaN,NaN,NaN
426,1061948,5487,Banants,7058,FC Santa Coloma,3,2,2014-07-08 14:00:00,Champions League,Andorra,...,2.0,NaN,8.0,1061691.0,NaN,NaN,NaN,NaN,NaN,NaN
434,1061691,7058,FC Santa Coloma,2997,Maccabi Tel Aviv,0,1,2014-07-15 18:00:00,Champions League,Israel,...,9.0,1061702.0,5.0,1061702.0,NaN,NaN,NaN,NaN,NaN,NaN
450,1061702,2997,Maccabi Tel Aviv,7058,FC Santa Coloma,2,0,2014-07-22 17:30:00,Champions League,Andorra,...,6.0,1061545.0,10.0,1061970.0,NaN,NaN,NaN,NaN,NaN,NaN


In [310]:
combined_fixtures_f[(combined_fixtures_f['cur_team_id'] == 8) & (combined_fixtures_f['opp_team_id'] == 2930)]

,index,cur_team_id,cur_team_score,fixture_id,opp_team_id,opp_team_score,starting_datetime,cur_team_order,cur_team_next,outcome,opp_team_order,opp_team_next,cur_team_r,opp_team_r,cur_p,opp_p,cur_post_r,opp_post_r
26,905229,8.0,2,1058774,2930.0,0,2008-02-19 19:45:00,1.0,1058782.0,1.0,1.0,1058782.0,1500.0,1500.0,0.500000,0.500000,1515.000000,1485.000000
27,910518,8.0,1,1058782,2930.0,0,2008-03-11 19:45:00,2.0,1058515.0,1.0,2.0,1058914.0,1515.0,1485.0,0.456934,0.543066,1501.291995,1498.708005


In [312]:
combined_fixtures_f[combined_fixtures_f['cur_team_id'] == 8]

,index,cur_team_id,cur_team_score,fixture_id,opp_team_id,opp_team_score,starting_datetime,cur_team_order,cur_team_next,outcome,opp_team_order,opp_team_next,cur_team_r,opp_team_r,cur_p,opp_p,cur_post_r,opp_post_r
26,905229,8.0,2,1058774,2930.0,0,2008-02-19 19:45:00,1.0,1058782.0,1.0,1.0,1058782.0,1500.000000,1500.0,0.500000,0.500000,1515.000000,1485.000000
27,910518,8.0,1,1058782,2930.0,0,2008-03-11 19:45:00,2.0,1058515.0,1.0,2.0,1058914.0,1515.000000,1485.0,0.456934,0.543066,1501.291995,1498.708005
28,916511,8.0,1,1058515,19.0,1,2008-04-02 18:45:00,3.0,1058523.0,0.5,3.0,1058523.0,1501.291995,1515.0,NaN,NaN,NaN,NaN
29,918441,8.0,4,1058523,19.0,2,2008-04-08 18:45:00,4.0,1058448.0,1.0,4.0,1060972.0,NaN,NaN,NaN,NaN,NaN,NaN
30,922762,8.0,1,1058448,18.0,1,2008-04-22 18:45:00,5.0,1058452.0,0.5,5.0,1058452.0,NaN,NaN,NaN,NaN,NaN,NaN
31,925006,8.0,2,1058452,18.0,3,2008-04-30 18:45:00,6.0,1061006.0,0.0,6.0,1058434.0,NaN,NaN,NaN,NaN,NaN,NaN
32,940076,8.0,0,1061006,700.0,0,2008-08-13 19:05:00,7.0,1061013.0,0.5,1.0,1061013.0,NaN,1500.0,NaN,NaN,NaN,NaN
33,943808,8.0,1,1061013,700.0,0,2008-08-27 19:05:00,8.0,1058872.0,1.0,2.0,1059018.0,NaN,NaN,NaN,NaN,NaN,NaN
34,949956,8.0,2,1058872,44.0,1,2008-09-16 18:45:00,9.0,1059078.0,1.0,3.0,1059070.0,NaN,NaN,NaN,NaN,NaN,NaN
35,954996,8.0,3,1059078,682.0,1,2008-10-01 18:45:00,10.0,1059219.0,1.0,2.0,1059211.0,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
#dupes = combined_fixtures[(combined_fixtures['fixture_order'] - combined_fixtures['fixture_order'].apply(np.floor)) > 0].sort_values('starting_datetime')`b
#dupes.to_csv('duplicate_fixtures')

In [92]:
fixtures = pd.merge(fixtures, combined_fixtures[['fixture_order', 'fixture_id', 'team_id']],\
                    how='left', left_on=['localteam_id','fixture_id'], right_on = ['team_id','fixture_id'])\
                .drop(['team_id'], axis=1)\
                .rename(index=str, columns={'fixture_order': 'localteam_order'})

In [93]:
fixtures = pd.merge(fixtures, combined_fixtures[['fixture_order', 'fixture_id', 'team_id']],\
                    how='left', left_on=['visitorteam_id','fixture_id'], right_on = ['team_id','fixture_id'])\
                .drop(['team_id'], axis=1)\
                .rename(index=str, columns={'fixture_order': 'visitorteam_order'})

In [94]:
fixtures = fixtures.sort_values('starting_datetime')

In [106]:
#assign initial rating to first game played by all teams
fixtures['localteam_rating'] = np.where(fixtures['localteam_order'] == 1, 1500, np.nan)
fixtures['visitorteam_rating'] = np.where(fixtures['visitorteam_order'] == 1, 1500, np.nan)

In [107]:
fixtures.head()

,fixture_id,localteam_id,localteam_name,visitorteam_id,visitorteam_name,localteam_score,visitorteam_score,starting_datetime,league_name,country_name,fifa_name,season_name,outcome,localteam_order,visitorteam_order,localteam_rating,visitorteam_rating
906067,22909,50,Torquay United,185,Exeter City,1,0,2008-01-01 12:00:00,Vanarama National League,England,"ENG,NIR,SCO,WAL",2007/2008,1.0,1.0,1.0,1500.0,1500.0
906074,654189,1244,DPMM FC,3347,Kedah,2,0,2008-01-01 12:45:00,Super League,Malaysia,MAS,2007/2008,1.0,1.0,1.0,1500.0,1500.0
906073,654188,1908,Negeri Sembilan,2276,Johor Darul Ta'zim,2,0,2008-01-01 12:45:00,Super League,Malaysia,MAS,2007/2008,1.0,1.0,1.0,1500.0,1500.0
906072,654187,3446,Perak,3398,Sarawak,5,0,2008-01-01 12:45:00,Super League,Malaysia,MAS,2007/2008,1.0,1.0,1.0,1500.0,1500.0
906068,5881,11,Fulham,18,Chelsea,1,2,2008-01-01 12:45:00,Premier League,England,"ENG,NIR,SCO,WAL",2007/2008,0.0,1.0,1.0,1500.0,1500.0


In [256]:
def Probability(rating1, rating2):
 
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))

In [109]:
def EloRating(Ra, Rb, K, d):

    # To calculate the Winning
    # Probability of Player A
    Pa = Probability(Rb, Ra)
 
    # To calculate the Winning
    # Probability of Player B
    Pb = Probability(Ra, Rb)
 
    # Case -1 When Player A wins
    # Updating the Elo Ratings
    if (d == 1) :
        Ra = Ra + K * (1 - Pa)
        Rb = Rb + K * (0 - Pb)
     
 
    # Case -2 When Player B wins
    # Updating the Elo Ratings
    else :
        Ra = Ra + K * (0 - Pa)
        Rb = Rb + K * (1 - Pb)
    
    return localteam_rating_post=Ra, visitorteam_rating_post=Rb
     
    ##    print("Updated Ratings:-")
    ##    print("Ra =", round(Ra, 6)," Rb =", round(Rb, 6))

In [111]:
test = fixtures[(fixtures['localteam_order'] == 1) & (fixtures['visitorteam_order'] == 1)]

In [115]:
test.apply(EloRating(test['localteam_rating'], test['visitorteam_rating'], 30, test['outcome']))

TypeError: cannot convert the series to <class 'float'>

In [120]:
test_onerow = test[0:1]

In [123]:
Probability(test_onerow['localteam_rating'], test_onerow['visitorteam_rating'])

0.5

In [128]:
test['p_localteam'] = test[['fixture_id', 'visitorteam_rating', 'localteam_rating']].apply(Probability)

TypeError: ("Probability() missing 1 required positional argument: 'rating2'", 'occurred at index fixture_id')

In [ ]:
def elo(data
# rank fixtures played by each team
    
# calcul


# Function to calculate the Probability
def Probability(rating1, rating2):
 
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))
 
 
# Function to calculate Elo rating
# K is a constant.
# d determines whether
# Player A wins or Player B. 

def EloRating(Ra, Rb, K, d):

    # To calculate the Winning
    # Probability of Player A
    Pa = Probability(Rb, Ra)
 
    # To calculate the Winning
    # Probability of Player B
    Pb = Probability(Ra, Rb)
 
    # Case -1 When Player A wins
    # Updating the Elo Ratings
    if (d == 1) :
        Ra = Ra + K * (1 - Pa)
        Rb = Rb + K * (0 - Pb)
     
 
    # Case -2 When Player B wins
    # Updating the Elo Ratings
    else :
        Ra = Ra + K * (0 - Pa)
        Rb = Rb + K * (1 - Pb)
     
 
    print("Updated Ratings:-")
    print("Ra =", round(Ra, 6)," Rb =", round(Rb, 6))
 
# Driver code
 
# Ra and Rb are current ELO ratings
Ra = 1200
Rb = 1000
K = 30
d = 1
EloRating(Ra, Rb, K, d)
 
# This code is contributed by
# Smitha Dinesh Semwal